# Preprocessing

# Feature exploring

In the first steps we explored which features are provided by the dataset and how they interact with eachother. One important feature group for us were the wind features. We found out, that wind is cooling down solar panels during the day and thus increasing their efficiency. Furthermore we found some articles that elaborate on the impact of the wind direction on the performance on solar panels and that it can be derived from the measured u and v components. So we decided to create the wind direction feature and also to estimate the wind speed with the u and v component. The calculated windspeed did not match up with the provided measured windspeed (possibly because of measurement inaccuracys) but it still increased the performance of our model.

We also explored a group of sun features (sun_azimuth and sun_elevation). With these features and information about the tilt angle and location of the solar panel, it is possible to derive information about the energy capture. Since we dont have information about the position or the tilt of the solar panel, we scraped the idea in the beginning.

During testing of our model, we also found out, that the features related to snow where harmful for the predictions. Therefore we removed them.
We also decided to remove constant features like the "wind speed w" and the "elevation", since they dont give us any relevant information.

Furthermore we looked into different individual features that were not intuitive for us. We researched on how they impact the solar energy production in the documentation and the internet:

Supercooled Liquid Water refers to liquid water droplets below the freezing point and can reduce the transmission of sunlight. It can be used to predict icing events.

visibility gives information about fog. If the visibility is below 1 km, the weather is declared as foggy. This is the reason we scaled it to kilometers for our model.




In [7]:
#import packages
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
# from Preprocessing import *

''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''
Preprocessing functions
'''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''
def drop_consecutive_nonzero_repeats(df):
    count = 0
    previous_value = None
    indices_to_drop = []

    for i, value in enumerate(df['pv_measurement']):
        if value != 0:  # Exclude zeros
            if value == previous_value:
                count += 1
                if count > 5:  # 3 consecutive times the same number
                    # Mark the indices to be dropped
                    indices_to_drop.extend(list(range(i - count, i + 1)))
            else:
                count = 0
            previous_value = value

    # Drop the rows with consecutive repeats
    df = df.drop(indices_to_drop)
    df.reset_index(drop=True, inplace=True)
    return df


def preprocessing(df,target,soort_data,id):
    
        target.rename(columns={'time': 'date_forecast'}, inplace=True)
        
        # 1. Attempt to drop the 'date_calc' column (if it exists) in each DataFrame
        if 'date_calc' in df.columns:
            df = df.drop('date_calc', axis=1)

        #print(df.head())

        imputer = IterativeImputer()

        # Select the numeric columns for imputation
        columns_to_impute = ['ceiling_height_agl:m','cloud_base_agl:m']

        # Perform imputation
        df[columns_to_impute] = imputer.fit_transform(df[columns_to_impute])

        #df['cloud_base_agl:m'].interpolate(method='linear', inplace=True)#.fillna(method='ffill', inplace=True)



        if id == 'a':
            df = df.drop("wind_speed_w_1000hPa:ms", axis=1)
            #df["visibility:m"] = df["visibility:m"] * 1000
            df = df.drop("elevation:m", axis=1)

            filter1 = [col for col in df.columns if 'snow' not in col or col in ['fresh_snow_24h:cm', 'fresh_snow_12h:cm']]
            df = df[filter1]

        if id == "b":

            df = df.drop("wind_speed_w_1000hPa:ms", axis=1)
            filter1 = [col for col in df.columns if 'snow' not in col or col in ['fresh_snow_24h:cm', 'fresh_snow_12h:cm']]
            df = df[filter1]

        if id == "c":
            df = df.drop("wind_speed_w_1000hPa:ms", axis=1)
            filter1 = [col for col in df.columns if 'snow' not in col or col in ['fresh_snow_24h:cm', 'fresh_snow_12h:cm', "snow_drift:idx"]]
            df = df[filter1]

        print("1")
        print(df.head())




            


                
    

       # 2. Linear interpolation for all columns
        #for features in df.columns:
        #    if features == 'snow_density:kgm3':
        #        df[features].fillna(0, inplace=True)
        #    else:
                # Interpolate missing values using linear interpolation
        #        df[features] = df[features].interpolate(method='linear')
        #print(df.shape)
        
        # # 3. Remove columns with constant data
        #unique_counts = df.nunique()
        #constant_features = unique_counts[unique_counts == 1].index
        #df = df.drop(columns=constant_features, axis=1)

        

        # 4. Set 'date_forecast' as the index and resample to hourly data

        df['date_forecast'] = pd.to_datetime(df['date_forecast'])
        df.set_index('date_forecast', inplace=True)
        df = df.resample('H').mean()
        #print(df.shape)

        # 5. Merge the first and second DataFrames in the preprocessed_features list 
        # 6. Merge the first and third DataFrames in the input_features list

        if soort_data == 'train_observed' or soort_data == 'train_estimated':
            df = pd.merge(df, target, on='date_forecast', how='inner')

        # 7. Conditional operations for 'pv_measurement' column (if it exists).
        if 'pv_measurement' in df.columns:
            # Define and use functions to drop consecutive non-zero and zero repeats (not provided)
            df = drop_consecutive_nonzero_repeats(df)
            #df = drop_consecutive_zero_repeats(df)
            #df = df.fillna(0)
            
         # 8. Attempt to drop the 'date_forecast' column (if it exists) in each DataFrame
        if 'date_forecast' in df.columns:
            df = df.drop('date_forecast', axis=1)
            
        # Drop all rows where all columns are empty
        if soort_data == 'train_observed' or soort_data == 'train_estimated':
            if id == "a":
                df = df.fillna(0)#dropna(how='any')
            else:
                df = df.fillna(0)
        else:
            df = df.dropna(how='all')
            #df = df.fillna(0)

        return df




'''''''''''''''''''''''''''''''''''''''''''''''''''''''''''
Functions
'''''''''''''''''''''''''''''''''''''''''''''''''''''''''''
#define function to load all the data
def load_data(location):
    target = pd.read_parquet(f'{location}/raw/train_targets.parquet')
    train_observed = pd.read_parquet(f'{location}/raw/X_train_observed.parquet')
    train_estimated = pd.read_parquet(f'{location}/raw/X_train_estimated.parquet')
    test_estimated = pd.read_parquet(f'{location}/raw/X_test_estimated.parquet')
    
    #put all the data of one location into a list
    data = [target, train_observed, train_estimated, test_estimated]
    return data


#preprocess the three different datasets for all locations
def preprocess_data(data,id):
    train_observed = preprocessing(data[1],data[0],'train_observed',id)
    train_estimated = preprocessing(data[2],data[0],'train_estimated',id)
    test_estimated = preprocessing(data[3],data[0],'test_estimated',id)
    data = [train_observed, train_estimated, test_estimated]
    return data


def save_to_file(data_to_file,location):
    #saving train estimated data to csv
    data_to_file[0].to_csv(f'{location}/preproc_train_observed_{location}.csv', index=False)
    data_to_file[1].to_csv(f'{location}/preproc_train_estimated_{location}.csv', index=False)
    data_to_file[2].to_csv(f'{location}/preproc_test_estimated_{location}.csv', index=False)
    



'''''''''''''''''''''''''''''''''''''''''''''''''''''''''''
Main pipline
'''''''''''''''''''''''''''''''''''''''''''''''''''''''''''



#load all the data and put them in a separate list for every location
data_A = load_data('A')
data_B = load_data('B')
data_C = load_data('C')

preprocessed_A = preprocess_data(data_A,"a")
preprocessed_B = preprocess_data(data_B,"b")
preprocessed_C = preprocess_data(data_C,"c")

#save preprocessed data
preprocessed_A = save_to_file(preprocessed_A,'A')
preprocessed_B = save_to_file(preprocessed_B,'B')
preprocessed_C = save_to_file(preprocessed_C,'C')

1
        date_forecast  absolute_humidity_2m:gm3  air_density_2m:kgm3  \
0 2019-06-02 22:00:00                       7.7                1.230   
1 2019-06-02 22:15:00                       7.7                1.229   
2 2019-06-02 22:30:00                       7.7                1.228   
3 2019-06-02 22:45:00                       7.7                1.226   
4 2019-06-02 23:00:00                       7.7                1.225   

   ceiling_height_agl:m  clear_sky_energy_1h:J  clear_sky_rad:W  \
0           1744.900024                    0.0              0.0   
1           1734.000000                    0.0              0.0   
2           1723.500000                    0.0              0.0   
3           1713.400024                    0.0              0.0   
4           1703.599976                    0.0              0.0   

   cloud_base_agl:m  dew_or_rime:idx  dew_point_2m:K  diffuse_rad:W  ...  \
0       1744.900024              0.0      280.299988            0.0  ...   
1       17

# Model Training

In [ ]:

from autogluon.tabular import TabularDataset, TabularPredictor

import pandas as pd
from itertools import combinations
import math
import numpy as np



'''''''''''''''''''''''''''''''''''''''''''''''''''''''''''
Functions
'''''''''''''''''''''''''''''''''''''''''''''''''''''''''''

def delete_snow(df):
    filter = [col for col in df.columns if "snow" not in col]
    df = df[filter]
    return df

def create_wind(df):
    pd.options.mode.chained_assignment = None
    epsilon = 1e-6

    df["windSpeed"] = np.sqrt(df["wind_speed_u_10m:ms"]**2 + df["wind_speed_v_10m:ms"]**2)
    df["windAngle"] = np.arctan2(df["wind_speed_v_10m:ms"], df["wind_speed_u_10m:ms"])
    return df

def create_time_feature(df):
    df['hourofday'] = df['date_forecast'].dt.hour
    df['dayofmonth'] = df['date_forecast'].dt.day
    df['month'] = df['date_forecast'].dt.month
    df['year'] = df['date_forecast'].dt.year
    df['dayofweek'] = df['date_forecast'].dt.dayofweek
    df['dayofyear'] = df['date_forecast'].dt.dayofyear
    return df


def create_submission(predictions_A,predictions_B,predictions_C):

    combined_predictions = pd.concat([predictions_A,predictions_B,predictions_C], axis=0)
    combined_predictions = combined_predictions.reset_index(drop=True)
    combined_predictions.index.name = 'id'
    combined_predictions.rename('prediction', inplace=True)
    # combined_predictions = combined_predictions.drop('Unnamed: 0', axis=1)
    combined_predictions.to_csv('auto_predictions_finalv31_bare_50_80.csv')


pd.options.mode.chained_assignment = None

train_data = TabularDataset('A/preproc_train_observed_A.csv')
train_data.head()
label = 'pv_measurement'
test_data = TabularDataset(f'A/preproc_train_estimated_A.csv')

#transform date
train_data['date_forecast'] = pd.to_datetime(train_data.date_forecast, format='%Y-%m-%d %H:%M:%S')
test_data['date_forecast'] = pd.to_datetime(test_data.date_forecast, format='%Y-%m-%d %H:%M:%S')

#create/delete features
train_data=create_time_feature(train_data)
test_data=create_time_feature(test_data)

train_data = delete_snow(train_data)
test_data = delete_snow(test_data)

train_data = create_wind(train_data)
test_data = create_wind(test_data)


#Add validation data to training weather is_est
percentage = 0.50
num_rows = int(len(test_data) * percentage)
sample = test_data.sample(n=num_rows, random_state=42)
test_data = test_data.drop(sample.index)
sample["is_estimated"] = pd.Series([1] * len(sample), name='is_estimated')
train_data["is_estimated"] = pd.Series([0] * len(train_data), name='is_estimated')

train_data = pd.concat([train_data,sample]).reset_index(drop=True)


#Train and predict
predictor = TabularPredictor(label=label, eval_metric='mean_absolute_error').fit(train_data, tuning_data=test_data)

#only for analysis. Takes some time to compute.
#x = predictor.feature_importance(test_data)
#print(x)


submission_data = TabularDataset('A/preproc_test_estimated_A.csv')
submission_data['date_forecast'] = pd.to_datetime(submission_data.date_forecast, format='%Y-%m-%d %H:%M:%S')

submission_data=create_time_feature(submission_data)
submission_data = create_wind(submission_data)
submission_data = delete_snow(submission_data)



predictions_A = predictor.predict(submission_data)

##########B

train_data = TabularDataset('B/preproc_train_observed_B.csv')
label = 'pv_measurement'
test_data = TabularDataset(f'B/preproc_train_estimated_B.csv')

#Add validation data to training
percentage = 0.50
num_rows = int(len(test_data) * percentage)
sample = test_data.sample(n=num_rows, random_state=42)
test_data = test_data.drop(sample.index)
train_data = pd.concat([train_data,sample]).reset_index(drop=True)

predictor = TabularPredictor(label=label, eval_metric='mean_absolute_error').fit(train_data, presets='best_quality', ag_args_fit={'num_gpus':1}, num_stack_levels=0,tuning_data=test_data,use_bag_holdout=True)
x = predictor.feature_importance(test_data)

print(x)

submission_data = TabularDataset('B/preproc_test_estimated_B.csv')

predictions_B = predictor.predict(submission_data)

###########C

train_data = TabularDataset('C/preproc_train_observed_C.csv')
label = 'pv_measurement'
test_data = TabularDataset(f'C/preproc_train_estimated_C.csv')
#train_data = pd.concat([train_data,test_data],ignore_index=True)

percentage = 0.50
num_rows = int(len(test_data) * percentage)
sample = test_data.sample(n=num_rows, random_state=42)
test_data = test_data.drop(sample.index)
train_data = pd.concat([train_data,sample]).reset_index(drop=True)



predictor = TabularPredictor(label=label, eval_metric='mean_absolute_error').fit(train_data, presets='best_quality', ag_args_fit={'num_gpus':1}, num_stack_levels=0,tuning_data=test_data,use_bag_holdout=True)
x = predictor.feature_importance(test_data)

print(x)

submission_data = TabularDataset('C/preproc_test_estimated_C.csv')

predictions_C = predictor.predict(submission_data)


create_submission(predictions_A,predictions_B, predictions_C)

        









